In [ ]:
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from tensorflow import keras
import gc
from sys import getsizeof
import datetime
import random

In [ ]:
#base_labels = ['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']
base_labels = ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone', 
                 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar','keyboard', 'lamp',
                 'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant']#, 
#                 'radio', 'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 
#                 'vase', 'wardrobe', 'xbox']
dim = 24
no_classes = len(base_labels)

training = np.zeros((0, dim, dim, dim))
test = np.zeros((0, dim, dim, dim))
training_labels = []
test_labels = []

for i, label in enumerate(base_labels): 
    file = './ModelNet40/output/' + label + '/output-train-' + str(dim) + '.h5'
#    file = './ModelNet10/ModelNet10/' + label + '/filled-train.h5'
    print('Procesing file ' + file + str(i))
    print(os.path.getsize(file))
    f = h5py.File(file, 'r')
    training = np.concatenate((training, f['tensor']))
    #print(tensor == 1)
    #training = np.concatenate((training, f['tensor'] == 1 ))
    print("trainin len " + str(getsizeof(training)))
    training_labels = np.concatenate((training_labels, np.full((len(f['tensor'])), i)))
    f.close()
    
    #test_file = './ModelNet40/output/' + label + '/output-test-' + label + '-48.h5'
    #print('Procesing file ' + test_file + str(i))
    #print(os.path.getsize(test_file))
    #f = h5py.File(test_file, 'r')
    #print(len(f['tensor']))
    #test = np.concatenate((test, f['tensor']))
    #test_labels = np.concatenate((test_labels, np.full((len(f['tensor'])), i)))
    #f.close()
    gc.collect()

In [ ]:
#model = keras.Sequential([
#    keras.layers.Flatten(input_shape=(32, 32, 32)),
#    keras.layers.Dense(128, activation='relu'),
#    keras.layers.Dense(10, activation='softmax')
#])
# Create the model
model = keras.Sequential()
model.add(
    keras.layers.Conv3D(
        dim, 
        kernel_size=(3), 
        activation='relu', 
        kernel_initializer='he_uniform', 
        data_format="channels_last",
        input_shape=(dim, dim, dim, no_classes)
    )
)

model.add(keras.layers.MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(keras.layers.Conv3D(128, kernel_size=(4, 4, 4), activation='relu', kernel_initializer='he_uniform'))
#model.add(keras.layers.Conv3D(160, kernel_size=(4, 4, 4), activation='relu', kernel_initializer='he_uniform'))
#model.add(keras.layers.Conv3D(48, kernel_size=(4, 4, 4), activation='relu', kernel_initializer='he_uniform'))
#model.add(keras.layers.MaxPooling3D(pool_size=(2, 2, 2)))
model.add(keras.layers.Flatten())
#model.add(keras.layers.Dense(8, activation='relu', kernel_initializer='he_uniform'))
model.add(keras.layers.Dense(no_classes, activation='softmax'))


#model.compile(optimizer='adam',
#              loss='sparse_categorical_crossentropy',
#              metrics=['accuracy'])


learning_rate = 0.001

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=learning_rate),
              metrics=['accuracy'])
print(model.summary())


In [ ]:
#model.fit(training, training_labels, epochs=200)

batch_size = 50
no_epochs = 10
learning_rate = 0.001

validation_split = 0.2
verbosity = 1
targets_train = keras.utils.to_categorical(training_labels)

history = model.fit(
                keras.utils.to_categorical(training, no_classes), 
                targets_train,
                batch_size=batch_size,
                epochs=no_epochs,
                verbose=verbosity,
                validation_split=validation_split,
                callbacks=[tensorboard_callback])


In [ ]:
test_loss, test_acc = model.evaluate(
    keras.utils.to_categorical(test, no_classes),  
    keras.utils.to_categorical(test_labels).astype(np.integer), verbose=2)
print('\nTest accuracy:', test_acc)

In [ ]:
len(training[0])

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(dim, dim, dim)),
    keras.layers.Dense(dim*2, activation='relu'),
    keras.layers.Dense(dim, activation='relu'),
    keras.layers.Dense(no_classes, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())


In [ ]:


log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


model.fit(training, 
          training_labels, 
          epochs=20,
          callbacks=[tensorboard_callback])


In [ ]:
test_loss, test_acc = model.evaluate(test,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

In [ ]:
index = random.randint(0, len(test))

predicted_index = np.argmax(p[index])
expected = base_labels[int(test_labels[index])]
predicted = base_labels[predicted_index]
draw(test[index], expected, predicted)
index

In [ ]:
for i, label in enumerate(base_labels): 
    
    file = './ModelNet40/output/' + label + '/output-test-' + str(dim)+ '.h5'
    #file = './ModelNet10/ModelNet10/' + label + '/filled-train.h5'
    print('Procesing file ' + file + str(i))
    print(os.path.getsize(file))
    f = h5py.File(file, 'r')
    print(len(f['tensor']))
    test = np.concatenate((test, f['tensor']))
    test_labels = np.concatenate((test_labels, np.full((len(f['tensor'])), i)))
    f.close()
    gc.collect()

In [ ]:
per_row = 3
num_figs = 6
cols = num_figs // per_row
fig, ax= plt.subplots(cols, per_row, subplot_kw={'projection': '3d'}, figsize=[15, 3 * cols])    

for i in range(0,num_figs):
    cube1 = (test[i+100][:,:,:] >= 1)
    ax[i//per_row, i%per_row].voxels(cube1, facecolors="blue")

plt.show()

In [ ]:
def draw(img, expected = '', predicted = ''):
    fig = plt.figure(figsize=[10,10])
    ax = fig.gca(projection='3d')
    cube1 = (img[:,:,:] >= 1)
    ax.voxels(cube1, facecolors="blue")
    plt.title('Expected: {}\n Predicted: {}'.format(expected, predicted), y=-0.01)
    plt.show()